In [16]:
# Load ground truth
ground_truth = pd.read_csv('./test_data/midlevel.chunks_90.split_0.train.csv')

# Parse the predictions.log file
predictions = []
with open('predictions.log', 'r') as file:
    for line in file:
        try:
            # Split the line to extract frame and activity information
            parts = line.strip().split(' - ')  # Splitting by " - " to separate timestamp and frame info
            frame_part = parts[1].split(': ')[0]  # Extract "Frame X"
            frame = int(frame_part.split()[1])  # Extract the frame number after "Frame"

            # Extract activity and confidence
            activity_part = parts[1].split(': ', maxsplit=1)[1]  # This contains "Predicted Activity: XYZ, Confidence: XX.XX%"
            activity = activity_part.split(', Confidence')[0].replace("Predicted Activity: ", "").strip()  # Extract actual activity
            
            # Extract confidence if it exists
            confidence = 0.0  # Default confidence
            if "Confidence" in activity_part:  # Check if "Confidence" exists in the string
                confidence = float(activity_part.split('Confidence: ')[1].strip('%'))  # Extract confidence as float

            # Append to predictions list
            predictions.append({'frame': frame, 'activity': activity, 'confidence': confidence})
        
        except (IndexError, ValueError) as e:
            print(f"Skipping line due to error: {line.strip()} -> {e}")

# Convert predictions to DataFrame
predictions_df = pd.DataFrame(predictions)

# Display the predictions DataFrame
print(predictions_df.head())


   frame              activity  confidence
0      0  opening_door_outside       53.74
1      1  opening_door_outside       54.08
2      2  opening_door_outside       57.44
3      3  opening_door_outside       57.44
4      4  opening_door_outside       57.44


In [17]:
def convert_predictions_to_segments(predictions):
    segments = []
    current_activity = None
    current_start = None

    for idx, row in predictions.iterrows():
        frame = row['frame']
        activity = row['activity']

        # Start a new segment if the activity changes
        if activity != current_activity:
            if current_activity is not None:
                # Save the previous segment
                segments.append({
                    'frame_start': current_start,
                    'frame_end': frame - 1,
                    'activity': current_activity
                })
            # Start a new segment
            current_activity = activity
            current_start = frame

    # Save the last segment
    if current_activity is not None:
        segments.append({
            'frame_start': current_start,
            'frame_end': predictions.iloc[-1]['frame'],
            'activity': current_activity
        })

    return pd.DataFrame(segments)

# Example: Convert frame-level predictions into segments
segments_df = convert_predictions_to_segments(predictions_df)

# Display the segments
print(segments_df)
segments_df.to_csv('segmented_activities.csv', index=False)

     frame_start  frame_end               activity
0              0         54   opening_door_outside
1             55         90   closing_door_outside
2             91        136   opening_door_outside
3            137        157           entering_car
4            158        184    closing_door_inside
..           ...        ...                    ...
323        18886      18904    fastening_seat_belt
324        18905      18945  unfastening_seat_belt
325        18946      18978    opening_door_inside
326        18979      19012            exiting_car
327        19013      19070   closing_door_outside

[328 rows x 3 columns]


In [18]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
file_path = './test_data/midlevel.chunks_90.split_0.train.csv'
data = pd.read_csv(file_path)

# Extract the unique activities from the 'activity' column
unique_activities = data['activity'].dropna().unique()

# Convert to a list
unique_activities_list = list(unique_activities)

# Print the unique activities
print(unique_activities_list)

['closing_door_outside', 'opening_door_outside', 'entering_car', 'closing_door_inside', 'fastening_seat_belt', 'using_multimedia_display', 'sitting_still', 'pressing_automation_button', 'fetching_an_object', 'opening_laptop', 'working_on_laptop', 'interacting_with_phone', 'closing_laptop', 'placing_an_object', 'unfastening_seat_belt', 'putting_on_jacket', 'opening_bottle', 'drinking', 'closing_bottle', 'looking_or_moving_around (e.g. searching)', 'preparing_food', 'eating', 'taking_off_sunglasses', 'putting_on_sunglasses', 'reading_newspaper', 'writing', 'talking_on_phone', 'reading_magazine', 'taking_off_jacket', 'opening_door_inside', 'exiting_car', 'opening_backpack', 'putting_laptop_into_backpack', 'taking_laptop_from_backpack']


In [21]:
def evaluate_multiclass(ground_truth, predictions, activity_classes):
    # Initialize metrics for each activity
    metrics = {cls: {'tp': 0, 'fp': 0, 'fn': 0} for cls in activity_classes}
    matched_chunks = set()  # Tracks matched ground truth chunks (annotation_id, chunk_id)

    # Iterate over predicted segments
    for _, pred in predictions.iterrows():
        pred_start = pred['frame_start']
        pred_end = pred['frame_end']
        pred_activity = pred['activity']

        # Calculate the midpoint of the prediction
        pred_midpoint = (pred_start + pred_end) / 2

        match_found = False
        for _, gt in ground_truth.iterrows():
            chunk_key = (gt['annotation_id'], gt['chunk_id'])  # Unique identifier for ground truth chunks

            # Check if the prediction matches this ground truth chunk
            if gt['activity'] == pred_activity and gt['frame_start'] <= pred_midpoint <= gt['frame_end']:
                if chunk_key not in matched_chunks:
                    metrics[pred_activity]['tp'] += 1  # True Positive
                    matched_chunks.add(chunk_key)  # Mark this chunk as matched
                    match_found = True
                    break
                else:
                    metrics[pred_activity]['fp'] += 1  # False Positive for duplicate detection
                    match_found = True

        if not match_found:
            if pred_activity in metrics:
                metrics[pred_activity]['fp'] += 1  # False Positive for unmatched prediction

    # Count False Negatives (ground truth chunks without a match)
    for _, gt in ground_truth.iterrows():
        chunk_key = (gt['annotation_id'], gt['chunk_id'])
        if chunk_key not in matched_chunks:
            metrics[gt['activity']]['fn'] += 1

    # Calculate Precision, Recall for each activity class
    precision, recall = {}, {}
    for cls in activity_classes:
        tp = metrics[cls]['tp']
        fp = metrics[cls]['fp']
        fn = metrics[cls]['fn']

        precision[cls] = tp / (tp + fp) * 100 if (tp + fp) > 0 else 0  # Convert to percentage
        recall[cls] = tp / (tp + fn) * 100 if (tp + fn) > 0 else 0  # Convert to percentage

    # Calculate overall Precision and Recall
    overall_tp = sum(metrics[cls]['tp'] for cls in activity_classes)
    overall_fp = sum(metrics[cls]['fp'] for cls in activity_classes)
    overall_fn = sum(metrics[cls]['fn'] for cls in activity_classes)

    overall_precision = overall_tp / (overall_tp + overall_fp) * 100 if (overall_tp + overall_fp) > 0 else 0
    overall_recall = overall_tp / (overall_tp + overall_fn) * 100 if (overall_tp + overall_fn) > 0 else 0

    return precision, recall, overall_precision, overall_recall

# Example usage:
activity_classes = unique_activities_list
precision, recall, overall_precision, overall_recall = evaluate_multiclass(ground_truth, segments_df, activity_classes)

print("Precision per class (%):", {cls: f"{precision[cls]:.2f}%" for cls in precision})
print("Recall per class (%):", {cls: f"{recall[cls]:.2f}%" for cls in recall})
print(f"Overall Precision: {overall_precision:.2f}%")
print(f"Overall Recall: {overall_recall:.2f}%")


Precision per class (%): {'closing_door_outside': '50.00%', 'opening_door_outside': '100.00%', 'entering_car': '100.00%', 'closing_door_inside': '50.00%', 'fastening_seat_belt': '50.00%', 'using_multimedia_display': '69.57%', 'sitting_still': '66.04%', 'pressing_automation_button': '52.63%', 'fetching_an_object': '75.00%', 'opening_laptop': '66.67%', 'working_on_laptop': '75.00%', 'interacting_with_phone': '62.50%', 'closing_laptop': '20.00%', 'placing_an_object': '41.25%', 'unfastening_seat_belt': '50.00%', 'putting_on_jacket': '26.67%', 'opening_bottle': '83.33%', 'drinking': '75.00%', 'closing_bottle': '80.00%', 'looking_or_moving_around (e.g. searching)': '27.27%', 'preparing_food': '26.67%', 'eating': '88.89%', 'taking_off_sunglasses': '33.33%', 'putting_on_sunglasses': '100.00%', 'reading_newspaper': '66.67%', 'writing': '50.00%', 'talking_on_phone': '100.00%', 'reading_magazine': '100.00%', 'taking_off_jacket': '100.00%', 'opening_door_inside': '33.33%', 'exiting_car': '100.00%'